# A2
**The notebook is with two model bi-directional and Attention-mechanism with simple lstm, the negative sampling and inference code is taken from chat gpt**

In [393]:
# import required libraries
import torchtext
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import sent_tokenize
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.nn.utils.rnn import pack_padded_sequence

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import collections
import math
from tqdm import tqdm
# !pip install -U torchtext==0.6   


In [2]:
# device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
# print(device, torch.cuda.get_device_name(0))
device = "cpu"

**Switched to cpu since gpu with smaller sample size take lot of time**

**The book dataset was taken from https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books**

In [339]:
with open('books/Book1.txt', encoding="utf8") as f:
    book1 = f.read()

In [349]:
#remove extra space
book1 = re.sub(r'(Page\s*\d+)', '' , book1)
book1 = re.sub(r'\s+', ' ', book1).strip()
# split text in sentences
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('punkt_tab')
book1_split = sent_tokenize(book1)
for i, sentence in enumerate(book1_split[:20]):
    print(f"#{i}: {sentence}")

#0: / THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
#1: They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.
#2: Mr. Dursley was the director of a firm called Grunnings, which made drills.
#3: He was a big, beefy man with hardly any neck, although he did have a very large mustache.
#4: Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.
#5: The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere.
#6: The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.
#7: They didn’t think they could bear it if anyone found out about the Potters.
#8: Mrs. Potter was Mrs

In [392]:
def process_merge():
    re_ = {"sentence" : [],"Book" : []}
    
    for i in range(1, 7):
        with open(f"{'books/Book'}{i}.txt", encoding="utf8") as f:
            book = f.read()
        #clean the white spaces and numbers
        book = re.sub(r'(Page\s*\d+)', '' , book)
        book = re.sub(r'\s+', ' ', book).strip()
        _split = sent_tokenize(book)
        
        idx = np.repeat(i, len(_split))
        re_["sentence"].extend(_split)
        re_['Book'].extend(idx)
        
    df = pd.DataFrame(re_)

    return df

In [386]:
# create the dataframe
df = process_merge()

In [387]:
# see the first 10 rows of the dataframe to validate they match the first 10 sentences of the first book
df.head(10)

,sentence,Book
0,"/ THE BOY WHO LIVED Mr. and Mrs. Dursley, of n...",1
1,They were the last people you’d expect to be i...,1
2,Mr. Dursley was the director of a firm called ...,1
3,"He was a big, beefy man with hardly any neck, ...",1
4,Mrs. Dursley was thin and blonde and had nearl...,1
5,The Dursley s had a small son called Dudley an...,1
6,"The Dursleys had everything they wanted, but t...",1
7,They didn’t think they could bear it if anyone...,1
8,"Mrs. Potter was Mrs. Dursley’s sister, but the...",1
9,The Dursleys shuddered to think what the neigh...,1


## preprocessing

In [388]:
#1. tokenization
corpus = df['sentence']
corpus = [sent.split(" ") for sent in corpus]
df['sentence'] = corpus

In [389]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def rpuncst(x):
     # Remove punctuation and stopwords
    # tokens = [token.lower() for token in tokens]
    # Apply stemming to each token
    tokens = [token for token in x if token.isalnum() and token.lower() not in stop_words]    
    return tokens

corpus = df['sentence']
corpus=corpus.apply(lambda x: rpuncst(x))

In [390]:
corpus

0        [BOY, LIVED, number, Privet, proud, say, perfe...
1        [last, people, expect, involved, anything, str...
2                  [Dursley, director, firm, called, made]
3                    [beefy, man, hardly, although, large]
4        [Dursley, thin, blonde, nearly, twice, usual, ...
                               ...                        
52619                                     [whatever, said]
52620    [going, come, round, mum, house, anything, eve...
52621                                         [miss, said]
52622    [hand, closed, automatically, around, fake, sp...
52623    [Page, 730, Harry, Potter, Half, Blood, Prince...
Name: sentence, Length: 52624, dtype: object

In [87]:
#creating a dataset loader
#* split the text into the input sequence length 
# problem with this approach is that we cannot split more than the length of the document if it is 7 then not more than 7 length each document will split into
# further we create padding pas pack_padding_sequence

class SimpleTextDataset(Dataset):
    def __init__(self, vocab, corpus, seq_len):
    
        self.vocab = vocab
        self.corpus = corpus
        self.seq_len = seq_len

    def __len__(self):
        # Calculate the total number of subsequences
        total_len = 0
        for sentence in self.corpus:
            total_len += len(sentence) - self.seq_len  # Number of valid subsequences from each sentence
        return max(total_len, 0)  # Ensure it's non-negative
    
    def __getitem__(self, idx):
        # Find the corresponding sentence and index for the given idx
        total_len = 0
        for sentence in self.corpus:
            sentence_len = len(sentence)
            if total_len + sentence_len > idx:
                start_idx = idx - total_len
                end_idx = start_idx + self.seq_len
                
                # Ensure we don't go out of bounds if seq_len is greater than remaining sentence length
                text = sentence[start_idx:end_idx]
                target = sentence[start_idx + 1:end_idx + 1] if end_idx <= len(sentence) else sentence[start_idx + 1:] + [self.vocab.get('<pad>', 0)] * (end_idx - len(sentence))
                
                break
            total_len += sentence_len
        
        # Convert text and target to tensors using vocab
        text_tensor = torch.tensor([self.vocab.get(word, self.vocab['<unk>']) for word in text], dtype=torch.long)
        target_tensor = torch.tensor([self.vocab.get(word, self.vocab['<unk>']) for word in target], dtype=torch.long)
        
        return {'text': text_tensor, 'target': target_tensor}



    # def __getitem__(self, idx):
    #     text = self.texts[idx]
        
    #     # Convert tokens to indices based on the vocabulary
    #     token_ids = [self.vocab.get(token, self.vocab['<unk>']) for token in text]
        
    #     # Convert the token list to a PyTorch tensor
    #     token_tensor = torch.tensor(token_ids, dtype=torch.long)
        
    #     return {'text': token_tensor, 'length': len(token_tensor)}

# Vocabulary creation (map each word to a unique index)
def build_vocab(texts):
    counter = Counter()
    for text in texts:
        counter.update(text)  # Count word occurrences in the tokenized sentences
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(counter.items())}  # Starting index from 2
    vocab['<pad>'] = 0  # Padding token (if ever needed)
    vocab['<unk>'] = 1  # Unknown token
    return vocab

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def collate_fn(batch):
    # Sort the batch by sequence length (longest first)
    batch = sorted(batch, key=lambda x: len(x['text']), reverse=True)

    # Get the text and targets
    texts = [item['text'] for item in batch]
    targets = [item['target'] for item in batch]

    # Convert lengths to a tensor
    lengths_tensor = torch.tensor([len(x) for x in texts], dtype=torch.long)

    # Pad the sequences to make them all the same length
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)
    padded_targets = pad_sequence(targets, batch_first=True, padding_value=0)

    # Ensure target length matches input length after padding
    target_length = padded_texts.size(1)
    if padded_targets.size(1) < target_length:
        padded_targets = torch.cat([padded_targets, torch.zeros(len(batch), target_length - padded_targets.size(1)).long().to(padded_targets.device)], dim=1)

    # Pack the padded sequences to inform the LSTM of the actual lengths
    packed_input = pack_padded_sequence(padded_texts, lengths_tensor, batch_first=True, enforce_sorted=False)

    return {
        'text': packed_input,   # Return packed text sequences
        'target': padded_targets,   # Return padded target sequences (next token)
        'length': lengths_tensor  # Lengths of each sequence
    }


In [93]:

# Build the vocabulary from the tokenized sentences
vocab = build_vocab(corpus)

# Create the dataset (without padding)
dataset = SimpleTextDataset(vocab, corpus, seq_len=3)

# Create the DataLoader to load data in batches
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, collate_fn=collate_fn)

# # Iterate through the DataLoader
for batch in dataloader:
    print(batch['text'])
    # print(f"Batch lengths: \n{batch['length']}")
    


In [90]:
dataloader

## build some bidirectional model

In [19]:
#the negative sampling is taken help from chatGPT

class BiLSTM_model(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim, pad_idx, device, negative_samples=5):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        self.hidden_dim = hidden_dim  # Define hidden_dim here
        self.num_directions = 2  # BiLSTM means 2 directions (forward and backward)
        
        # BiLSTM with two directions (forward and backward)
        self.forward_lstm = nn.LSTMCell(embed_dim, hidden_dim)
        self.backward_lstm = nn.LSTMCell(embed_dim, hidden_dim)
        
        # Fully connected layer to map from hidden state to output dimension (vocab size)
        self.fc = nn.Linear(hidden_dim * self.num_directions, output_dim)
        
        # Set the number of negative samples (default to 5)
        self.negative_samples = negative_samples

    def forward(self, packed_text, text_lengths):
        # Unpack the sequence
        padded_texts, _ = nn.utils.rnn.pad_packed_sequence(packed_text, batch_first=True, padding_value=0)
        
        # Embedding layer: Get word embeddings for the sequence
        embedded = self.embedding(padded_texts)  # Shape: [batch_size, seq_len, embed_dim]
        
        batch_size, seq_len, _ = embedded.size()

        # Initialize hidden states for BiLSTM (both forward and backward directions)
        h_forward = torch.zeros(batch_size, self.hidden_dim).to(embedded.device)
        c_forward = torch.zeros(batch_size, self.hidden_dim).to(embedded.device)

        h_backward = torch.zeros(batch_size, self.hidden_dim).to(embedded.device)
        c_backward = torch.zeros(batch_size, self.hidden_dim).to(embedded.device)

        output_forward = []
        output_backward = []

        # Process forward LSTM
        for t in range(seq_len):
            h_forward, c_forward = self.forward_lstm(embedded[:, t, :], (h_forward, c_forward))
            output_forward.append(h_forward.unsqueeze(1))

        # Process backward LSTM
        for t in range(seq_len - 1, -1, -1):
            h_backward, c_backward = self.backward_lstm(embedded[:, t, :], (h_backward, c_backward))
            output_backward.append(h_backward.unsqueeze(1))

        # Concatenate forward and backward hidden states
        output_forward = torch.cat(output_forward, dim=1)
        output_backward = torch.cat(output_backward[::-1], dim=1)  # Reverse backward to match original order

        # Concatenate forward and backward outputs
        output = torch.cat((output_forward, output_backward), dim=-1)  # Shape: [batch_size, seq_len, hidden_dim * 2]
        
        # Final fully connected layer
        output = self.fc(output.contiguous().view(batch_size * seq_len, -1))  # Flatten for the FC layer
        
        return output.view(batch_size, seq_len, -1)  # Reshape to [batch_size, seq_len, output_dim]

    def negative_sampling_loss(self, output, target, vocab_size):
        
        batch_size, seq_len, _ = output.size()
        
        # Flatten the output and target for loss calculation
        output_flat = output.view(-1, output.size(-1))  # Shape: [batch_size * seq_len, output_dim]
        target_flat = target.view(-1)  # Shape: [batch_size * seq_len]

        # Positive sample score: Use the true target words
        positive_score = output_flat.gather(1, target_flat.unsqueeze(1))  # Shape: [batch_size * seq_len, 1]

        # Negative sampling: randomly sample negative words
        neg_samples = torch.randint(0, vocab_size, (batch_size * seq_len, self.negative_samples), device=output.device)

        # Get the scores for the negative samples
        negative_scores = output_flat.gather(1, neg_samples)  # Shape: [batch_size * seq_len, negative_samples]
        
        # Apply sigmoid to the positive and negative scores
        positive_score = torch.sigmoid(positive_score)
        negative_scores = torch.sigmoid(-negative_scores)

        # Compute the loss: Positive log-likelihood + Negative log-likelihood
        positive_loss = -torch.log(positive_score).sum()  # Sum over positive samples
        negative_loss = -torch.log(negative_scores).sum()  # Sum over negative samples
        
        # Total loss (positive + negative)
        loss = (positive_loss + negative_loss) / (batch_size * seq_len)

        return loss


In [105]:
#this is just to test the batch loader
embed_dim = 50
hidden_dim = 64
vocab_size = len(vocab)  # Example vocab size
output_dim = 2 * hidden_dim  # BiLSTM, so output_dim is doubled
pad_idx = 0  # Padding index (often 0)

# Initialize model
model = BiLSTM_model(vocab_size, embed_dim, hidden_dim, output_dim, pad_idx, device).to(device)

# Example input
for batch in dataloader:
    packed_input = batch['text'].to(device)
    text_lengths = batch['length'].to(device)

    # Forward pass
    output = model(packed_input, text_lengths)
    print(f"Output shape: {output.shape}")


Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size([10, 3, 128])
Output shape: torch.Size(

KeyboardInterrupt: 

In [63]:
def train(model, dataloader, optimizer, device, vocab_size, num_epochs=10):
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        epoch_loss = 0  # To track the loss over the epoch
        correct_predictions = 0  # Track the number of correct predictions
        total_predictions = 0  # Track the total number of predictions
        
        # Loop over the DataLoader
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False):
            # Get inputs and targets from the batch
            text = batch['text'].to(device)
            text_lengths = batch['length'].to(device)
            target = batch['target'].to(device)  # Make sure to provide the target in your dataset
            
            # Zero the gradients
            optimizer.zero_grad()

            output = model(text, text_lengths)
            
            # Compute the negative sampling loss
            loss = model.negative_sampling_loss(output, target, vocab_size)
            
            # Backward pass
            loss.backward()
            
            # Update the model parameters
            optimizer.step()

            # Track the loss
            epoch_loss += loss.item()

            # Calculate accuracy
            # Flatten output and target for easier comparison
            output_flat = output.view(-1, output.size(-1))  # Shape: [batch_size * seq_len, output_dim]
            target_flat = target.view(-1)  # Shape: [batch_size * seq_len]

            # Get predicted word indices (the index with the highest score)
            _, predicted_indices = output_flat.max(dim=1)  # Shape: [batch_size * seq_len]

            # Compare predicted indices with target indices
            correct_predictions += (predicted_indices == target_flat).sum().item()
            total_predictions += target_flat.size(0)
               # Compute average loss and accuracy

        # Compute average loss and accuracy
        avg_loss = epoch_loss / len(dataloader)
        accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0.0
        
        # Print the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    print("Training complete!")

# Example usage:
# Define your model, optimizer, loss function, etc.
vocab_size = 20031  # Example vocab size
embed_dim = 50
hidden_dim = 64
output_dim = vocab_size  # Output dimension should match vocab size for prediction tasks
pad_idx = 0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create the model
model = BiLSTM_model(vocab_size, embed_dim, hidden_dim, output_dim, pad_idx, device, negative_samples=5).to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Assuming `dataloader` is ready (make sure your dataloader gives you a 'target' for the next-token prediction)
train(model, dataloader, optimizer, device, vocab_size, num_epochs=10)


Epoch 1/10 - Loss: 0.7873, Accuracy: 28.64%


Epoch 2/10 - Loss: 0.3421, Accuracy: 38.68%


Epoch 3/10 - Loss: 0.2484, Accuracy: 50.61%


Epoch 4/10 - Loss: 0.2038, Accuracy: 56.94%


Epoch 5/10 - Loss: 0.1777, Accuracy: 59.52%


Epoch 6/10 - Loss: 0.1585, Accuracy: 62.39%


Epoch 7/10 - Loss: 0.1437, Accuracy: 63.94%


Epoch 8/10 - Loss: 0.1329, Accuracy: 64.41%


Epoch 9/10 - Loss: 0.1236, Accuracy: 65.12%


Epoch 10/10 - Loss: 0.1184, Accuracy: 65.59%
Training complete!


In [65]:
torch.save(model,'Bi-directional-lstm.pth')

## inference results of bi-direction-LSTM

In [20]:
model=torch.load('Bi-directional-lstm.pth', weights_only=False)
model.eval()

BiLSTM_model(
  (embedding): Embedding(20031, 50, padding_idx=0)
  (forward_lstm): LSTMCell(50, 64)
  (backward_lstm): LSTMCell(50, 64)
  (fc): Linear(in_features=128, out_features=20031, bias=True)
)

In [68]:
#part of the code is taken is from chat gpt
def infer_with_sliding_window(model, input_sentence, vocab, device, seq_len=3, pad_idx=0):
    
    model.eval()

    input_tensor = torch.tensor([vocab.get(word, vocab['<unk>']) for word in input_sentence], dtype=torch.long).to(device)
    
    # Pad the sequence to the required length (if necessary)
    if len(input_tensor) < seq_len:
        padding = torch.full((seq_len - len(input_tensor),), pad_idx, dtype=torch.long).to(device)
        input_tensor = torch.cat((input_tensor, padding), dim=0).to(device)

    # Sliding window: create chunks of size `seq_len` with stride of 1
    chunks = [input_tensor[i:i+seq_len] for i in range(0, len(input_tensor) - seq_len + 1)]
    
    predicted_words = []

    # For each chunk, predict the next word
    for chunk in chunks:
        # Pad the chunk if it's smaller than `seq_len` (i.e., the last chunk)
        if len(chunk) < seq_len:
            chunk = torch.cat((chunk, torch.full((seq_len - len(chunk),), pad_idx, dtype=torch.long).to(device)))

        # Create a tensor for the current chunk and its length
        chunk = chunk.unsqueeze(0)  # Add batch dimension (batch_size = 1)
        lengths = torch.tensor([len(chunk)], dtype=torch.long)
        
        # Pack the input sequence (for variable-length sequences)
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(chunk, lengths, batch_first=True, enforce_sorted=False).to(device)

        # Get the model's prediction for this chunk
        with torch.no_grad():
            output = model(packed_input, lengths)
        
        # Convert the output to a list of predicted tokens for each time step
        output = output.squeeze(0)  # Remove batch dimension (batch_size = 1)
        
        # For the last token in the chunk, predict the next word
        _, predicted_idx = output[-1].max(dim=-1)  # Get the predicted index for the last token
        predicted_word = list(vocab.keys())[list(vocab.values()).index(predicted_idx.item())]
        predicted_words.append(predicted_word)

    return predicted_words


In [77]:
# Example usage:
# Assuming the model is loaded, vocab is available, and device is set to 'cpu' or 'cuda'
input_sentence = ['gotten', 'terrible', 'trouble', 'found', 'roof', 'school']
 # 'go!' # Example input sentence (tokenized)
predicted_words = infer_with_sliding_window(model, input_sentence, vocab, "cuda:0", seq_len=4)

print(f"Predicted words: {predicted_words}")

Predicted words: ['said', 'terrible', 'Dad']


In [90]:
corpus.loc[40]

['Dursley', 'always', 'sat', 'back', 'window', 'office', 'ninth']

In [87]:
df.loc[40]['sentence']

['Mr.',
 'Dursley',
 'always',
 'sat',
 'with',
 'his',
 'back',
 'to',
 'the',
 'window',
 'in',
 'his',
 'office',
 'on',
 'the',
 'ninth',
 'floor.']

In [85]:
import torch

def infer_last_word(model, input_sentence, vocab, device, pad_idx=0):
   .    
    # Ensure the model is in evaluation mode
    model.eval()

    # Tokenize and pad the input sequence
    input_tensor = torch.tensor([vocab.get(word, vocab['<unk>']) for word in input_sentence], dtype=torch.long).to(device)
    
    # Pad the sequence to the required length (if necessary)
    # if len(input_tensor) < seq_len:
    #     padding = torch.full((seq_len - len(input_tensor),), pad_idx, dtype=torch.long).to(device)
    #     input_tensor = torch.cat((input_tensor, padding), dim=0).to(device)

    # Create tensor for the full input sentence
    input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension (batch_size = 1)
    lengths = torch.tensor([len(input_tensor)], dtype=torch.long)
    
    # Pack the input sequence (for variable-length sequences)
    packed_input = torch.nn.utils.rnn.pack_padded_sequence(input_tensor, lengths, batch_first=True, enforce_sorted=False).to(device)

    # Get the model's prediction for the entire input sequence
    with torch.no_grad():
        output = model(packed_input, lengths)
    
    # Get the output corresponding to the last token (the last word in the sentence)
    output = output.squeeze(0)  # Remove batch dimension (batch_size = 1)
    
    # For the last token in the sequence, predict the next word
    # We get the output for the last token position
    _, predicted_idx = output[-1].max(dim=-1)  # Get the predicted index for the last token
    
    # Convert the predicted index back to a word
    predicted_word = list(vocab.keys())[list(vocab.values()).index(predicted_idx.item())]
    
    return predicted_word


In [93]:
input_sentence = ['Dursley', 'always', 'sat', 'back', 'window', 'office', 'ninth']
 # 'go!' # Example input sentence (tokenized)
predicted_words = infer_last_word(model, input_sentence, vocab, "cuda:0")
print(predicted_words)


sipped


In [35]:
import torch
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader

def infer_random_sample(model, dataloader, criterion, device,seq_len=3):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradients needed for inference
        for batch in dataloader:
            packed_input = batch['text']  # PackedSequence input (already packed)
            target = batch['target']     # Target sequences
            lengths = batch['length']     # Length of each sequence in the batch
            
            # Get the total number of sequences in the batch
            batch_size = packed_input.batch_sizes[0]  # Number of sequences in the batch
            
            # Pick a random index for a sentence in the batch
            random_idx = random.randint(0, batch_size - 1)

            # Extract the lengths of the selected sample sequence
            seq_length = lengths[random_idx].item()

            # Unpack the packed input (so we can access individual sequences)
            padded_input, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_input, batch_first=True, padding_value=0)

            # Extract the random sample sequence (no padding)
            random_sample = padded_input[random_idx, :seq_length]  # Remove padding

            # Forward pass through the model with the random sample
            # We need to convert it back to a PackedSequence before passing through the model
            packed_sample = torch.nn.utils.rnn.pack_padded_sequence(random_sample.unsqueeze(0), [seq_length], batch_first=True, enforce_sorted=False)
            # Forward pass through the model
            output = model(packed_sample, [seq_length])  # Assuming seq_len is passed in as a list

            # Calculate the loss for this random sample
            loss = criterion(output.view(-1, output.size(-1)), target[random_idx].view(-1))  # Flatten for loss computation

            # Get predicted word indices (max over the output)
            _, predicted_indices = output.max(dim=-1)  # Get indices of the predicted tokens

            # Calculate accuracy for the random sample
            correct = (predicted_indices.squeeze() == target[random_idx]).sum().item()
            total = target[random_idx].numel()
            accuracy = 100 * correct / total

            print(f"Random Sample Loss: {loss.item():.4f}")
            print(f"Random Sample Accuracy: {accuracy:.2f}%")
            break  # We only need to compute for one random sample, break after first sample


In [36]:
!export CUDA_LAUNCH_BLOCKING=1
# torch.cuda.empty_cache()
model=model.cpu()
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index in loss computation
device ="cpu"
# Assuming you have a DataLoader `dataloader` prepared
infer_random_sample(model, dataloader, criterion, device=device,seq_len=3)

Random Sample Loss: 2.2029
Random Sample Accuracy: 33.33%


# Attension-mechansim 

In [270]:
#let us build a very simple attension mechanism for classification tasks
#the code is taken from the provided notebook
class LSTM_GAtt(nn.Module):
    def __init__(self, input_dim: int, embed_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        
        # let's use pytorch's LSTM
        self.lstm = nn.LSTM(embed_dim, 
                           hidden_dim, 
                           num_layers=4, 
                           bidirectional=True, 
                           dropout=.5,
                           batch_first=True)
        
        # Linear Layer for binary classification 
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def attention_net(self, lstm_output, hn):
        
        h_t      = hn.unsqueeze(2)
        H_keys   = torch.clone(lstm_output)
        H_values = torch.clone(lstm_output)
        
        alignment_score   = torch.bmm(H_keys, h_t).squeeze(2) # SHAPE : (bs, seq_len, 1)
        
        soft_attn_weights = F.softmax(alignment_score, 1) # SHAPE : (bs, seq_len, 1)
        
        context           = torch.bmm(H_values.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2) # SHAPE : (bs, hidden_size * num_directions)
        
        return context

    def forward(self, text, text_lengths):

        embedded = self.embedding(text) # SHAPE : (batch_size, seq_len, embed_dim)

        lstm_output, (hn, cn) = self.lstm(embedded)
        
        # This is how we concatenate the forward hidden and backward hidden from Pytorch's BiLSTM
        hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)

        attn_output = self.attention_net(lstm_output, hn)
        output = self.fc(attn_output)
        return output

In [40]:
#trying on our old corpus for classfication tasks

df=pd.read_parquet('0.parquet')
df['text'] = df['text'].astype('unicode')
corpus = df['text']
corpus = [sent.split(" ") for sent in corpus]
df['text'] = corpus

In [102]:
corpus = df[:1000]
corpus['text']=corpus['text'].apply(lambda x: rpuncst(x))

/tmp/ipykernel_2067273/1004626873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text']=corpus['text'].apply(lambda x: rpuncst(x))


In [103]:
corpus

,text,label
0,"[frustration, patient, repeat, experience, man...",0
1,"[going, Goldberg, 10, think, one, 1st, patient...",1
2,"[know, Goldberg, like, moving, let, tell, STAY...",0
3,"[writing, review, give, heads, see, office, st...",0
4,"[food, great, best, thing, wings, simply, best...",1
...,...,...
995,"[lot, bad, experiences, employees, rude, much,...",0
996,"[location, caters, predominantly, low, income,...",0
997,"[Barber, Shop, good, old, fashion, convo, spor...",1
998,"[problem, dry, cleaner, poor, counter, never, ...",0


In [45]:
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus['text']))) #al

In [46]:
word2index = {v:idx for idx, v in enumerate(vocabs)}
vocabs.append('<UNK>')
word2index['<UNK>'] = 0

In [104]:
list_seq=corpus['text'].tolist()
#we will pad to the maximum values to keep things simple

# pad_sequence(torch.tensor(list_seq)).size()
padding_mat = []
for sentence in list_seq:
    padding_mat.append([word2index[word] for word in sentence])
    

In [108]:
#create padding 
import torch

def pad_batch_to_fixed_length(batch, seq_len, pad_idx=0, device='cpu'):
    padded_batch = []

    for seq in batch:
        # Convert the sequence to a tensor
        seq_tensor = torch.tensor(seq, dtype=torch.long).to(device)

        # If sequence length is shorter than `seq_len`, pad it
        if seq_tensor.size(0) < seq_len:
            padding = torch.full((seq_len - seq_tensor.size(0),), pad_idx, dtype=torch.long).to(device)
            padded_seq = torch.cat((seq_tensor, padding), dim=0)
        else:
            padded_seq = seq_tensor[:seq_len]  # Truncate if longer than seq_len

        padded_batch.append(padded_seq)

    # Stack the padded sequences into a tensor
    padded_batch = torch.stack(padded_batch, dim=0)
    return padded_batch

# Example input (batch of tokenized sentences)

# Desired fixed sequence length
fixed_seq_len = 8
pad_idx = 0  # Padding index
device = 'cpu'  # Assuming you're using CPU

# Pad the batch
padded_batch = pad_batch_to_fixed_length(padding_mat, fixed_seq_len, pad_idx, device)

print("Padded batch:")
print(padded_batch)



Padded batch:
tensor([[6732, 4900, 7920,  ..., 5880, 2849, 4795],
        [ 335, 5255,  230,  ..., 4677, 2973, 1754],
        [5050, 5255, 5646,  ..., 5719, 6945, 7354],
        ...,
        [8601, 3541, 4795,  ..., 1072,  289,  280],
        [1067, 7998, 5682,  ..., 3418, 1047, 8176],
        [ 402, 4289, 7219,  ..., 5531, 6482, 2846]])


In [271]:
# input_dim = 

# LSTM_GAtt(input_dim, embed_dim, hidden_dim, output_dim):

input_dim = len(vocab)
embed_dim = 50
hidden_dim = 60 
output_dim = 1
model=LSTM_GAtt(input_dim, embed_dim, hidden_dim, output_dim)

In [272]:
model

LSTM_GAtt(
  (embedding): Embedding(8744, 50, padding_idx=0)
  (lstm): LSTM(50, 60, num_layers=4, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=120, out_features=1, bias=True)
)

In [273]:
# Use BCEWithLogitsLoss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn  = nn.BCEWithLogitsLoss()


for epoch in range(epochs):
    epoch_loss = 0  # To track the loss over the epoch
    correct_predictions = 0  # Track the number of correct predictions
    total_predictions = 0  # Track the total predictions
    
    # Iterate over each sequence in the padded_batch
    for step_counter in tqdm(range(padded_batch.size(0))):  # Iterate through each row (sequence)
        optimizer.zero_grad()  # Zero out gradients before the forward pass
        
        # Extract the current sequence and its corresponding label
        batch_tensor = padded_batch[step_counter].unsqueeze(0).long().to(device)  # Add batch dimension
        label = torch.tensor(corpus['label'].loc[step_counter], dtype=torch.float).to(device)  # Get the label for this sequence

        # Get the length of the sequence (this should be the same across the batch)
        seq_len = batch_tensor.size(1)  # The length of the sequence for this particular example

        # Forward pass through the model
        output = model(batch_tensor, torch.tensor([seq_len]).to(device))  # Pass through the model
        
        # No need to apply sigmoid here since BCEWithLogitsLoss expects raw logits
        loss = loss_fn(output.squeeze(), label)  # Apply loss function directly on logits
        
        epoch_loss += loss.item()
        
        # Backpropagate and update weights
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy (sigmoid the output and compare with threshold)
        output_prob = torch.sigmoid(output).squeeze()  # Apply sigmoid to get probabilities
        
        # If probability > 0.5, we predict class 1, else class 0
        prediction = (output_prob > 0.5).float()
        
        # Update correct predictions and total predictions
        correct_predictions += (prediction == label).sum().item()
        total_predictions += 1 # Since batch size is 1, total_predictions is incremented by 1 each time
        
    # Calculate average loss and accuracy for the epoch
    avg_loss = epoch_loss / padded_batch.size(0)  # Average loss over the epoch
    accuracy = correct_predictions / total_predictions  # Accuracy for the epoch
    
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")


100%|██████████| 1000/1000 [00:35<00:00, 28.04it/s]


Epoch 1/10 - Loss: 0.6899, Accuracy: 54.80%


100%|██████████| 1000/1000 [00:34<00:00, 28.75it/s]


Epoch 2/10 - Loss: 0.6843, Accuracy: 57.10%


100%|██████████| 1000/1000 [00:35<00:00, 27.85it/s]


...

Epoch 8/10 - Loss: 0.3739, Accuracy: 84.50%


100%|██████████| 1000/1000 [00:33<00:00, 29.77it/s]


Epoch 9/10 - Loss: 0.3200, Accuracy: 87.90%


100%|██████████| 1000/1000 [00:34<00:00, 29.24it/s]

Epoch 10/10 - Loss: 0.2591, Accuracy: 90.80%


In [278]:
#random sample
sample=np.array(np.round(np.random.uniform(0,len(padding_mat),10)), dtype ='int')
text=np.array(padded_batch)[sample]
lable=np.array(corpus['label'])[sample]

In [262]:
padded_batch.size(0)

1000

In [323]:
res = {'prediction':[], 'label':[],'text':[]}
for i,t in enumerate(text):
    te = torch.tensor(t, dtype=torch.long).unsqueeze(0)
    size_ = te.size(1)
    with torch.no_grad():
        output = model(te, size_)
        loss = loss_fn(output.squeeze().float(), torch.tensor(lable[i], dtype=torch.float))  # Apply loss function directly on logits
        output_prob = torch.sigmoid(output).squeeze()  # Apply sigmoid to get probabilities
        prediction = (output_prob > 0.5).float()
        res['prediction'].append(prediction)
        res['label'].append(lable[i])
        res['text'].append(df['text'].loc[i])
        

In [310]:
for k,r in enumerate(res['text']):
    res['text'][k]=''.join(r)

In [325]:
pd.DataFrame(res)

,prediction,label,text
0,tensor(1.),1,"[Unfortunately,, the, frustration, of, being, ..."
1,tensor(1.),1,"[Been, going, to, Dr., Goldberg, for, over, 10..."
2,tensor(0.),0,"[I, don't, know, what, Dr., Goldberg, was, lik..."
3,tensor(1.),1,"[I'm, writing, this, review, to, give, you, a,..."
4,tensor(0.),0,"[All, the, food, is, great, here., But, the, b..."
5,tensor(0.),1,"[Wing, sauce, is, like, water., Pretty, much, ..."
6,tensor(1.),0,"[Owning, a, driving, range, inside, the, city,..."
7,tensor(0.),0,"[This, place, is, absolute, garbage..., , Half..."
8,tensor(0.),0,"[Before, I, finally, made, it, over, to, this,..."
9,tensor(0.),0,"[I, drove, by, yesterday, to, get, a, sneak, p..."


In [332]:
torch.save(model.state_dict(), "Attention_weights")
torch.save(model, "Attention_model")

# final comparision

In [394]:
from IPython.display import display, HTML
import pandas as pd
v=pd.DataFrame(columns =['Model', 'Traning loss',  'Accuracy', 'Task'])
v['Model'] = ['Bi-Directional-LSTM', 'Attention-lstm', 'Bi-Directional-50']
v['Traning loss'] = ['0.1192', '0.2591', ' 0.0568']
v['Accuracy'] = ['65', '90', '67']
v['Task'] = ['Next word prediction', 'Classfication', 'Next word prediction']
display(v)

,Model,Traning loss,Accuracy,Task
0,Bi-Directional-LSTM,0.1192,65,Next word prediction
1,Attention-lstm,0.2591,90,Classfication
2,Bi-Directional-50,0.0568,67,Next word prediction


** Observation and challanges **
* The Bi-directional-LSTM was hard to fit the model keeps on increasing the accuracy and loss tried various method to click gradient, then we tried negative sample
* inference with Bi-LSTM is done in two ways although the model is next work prediction rolling window size prediction which gives a bit sensable results and last wors prediction which is again worst in performance
* Tried to run the model for longer duration does not give any performance gain
* learning rate is reduced to 0.001
* Attention mechanism since produce the linear layer output in order to find the curitial word in a sentence to classify is intresting problem
* The part of inferenc code and negative sampling is taken help from chatgpt , but it was the self idea to try